# CNN - Block architecture
In this notebook we develop models using the standard block structure of convolutional neural networks. The points reinforced are the same as the one seen in the reference below. We also compare our results to those of standard models such as LeNet-5. **The end goal is to have all of these models in a .py so that they can be imported from other notebooks.**

Modifications should be around:
- the number of conv layers (depending on input resolution)
- the size of the conv kernels
- whether or not to use dropout layers (probably)
- which metrics to use (classification)
- what parameters to use for the optimizer 

See https://github.com/riblidezso/peak_steepness/blob/master/neural_network_predictions/nn_utils.py and https://arxiv.org/pdf/1806.05995.pdf
for references that use similarly "abstract" data although in a regression context.

### Importing libraries

In [13]:
import numpy as np
import time
# keras
import keras
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten,  AveragePooling2D, Conv2D
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
# scikit learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
# data processing pipeline - see .py file
from data_processing import processing_pipeline
# plotting
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.ndimage.filters import convolve
# for reloading
from importlib import reload

### Defining functions that return compiled model

In [4]:
# building the network on top of the one seen at https://arxiv.org/pdf/1806.05995.pdf
# their code: https://github.com/riblidezso/peak_steepness/blob/master/neural_network_predictions/nn_utils.py

def get_ribli_model():  
    model = Sequential()
    model.name = 'ribli'

    model.add(Conv2D(32, (3, 3), input_shape=(256, 256, 1), name='L01', activation='relu'))
    model.add(Conv2D(32, (3, 3), name='L02', activation='relu'))

    model.add(AveragePooling2D(pool_size=(2, 2), name='L03'))
    model.add(Conv2D(64, (3, 3), name='L04', activation='relu'))
    model.add(Conv2D(64, (3, 3), name='L05', activation='relu'))
    
    model.add(AveragePooling2D(pool_size=(2, 2), name='L06'))
    
    model.add(Conv2D(128, (3, 3), name='L07', activation='relu'))
    model.add(Conv2D(128, (3, 3), name='L08', activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), name='L09'))

    model.add(Conv2D(128, (3, 3), name='L10', activation='relu'))
    model.add(Conv2D(128, (3, 3), name='L11', activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), name='L12'))
    
    model.add(Conv2D(128, (3, 3), name='L13', activation='relu'))
    model.add(Conv2D(128, (3, 3), name='L14', activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), name='L15'))
    
    model.add(Flatten())
    model.add(Dense(256, name='L16', activation='relu'))
    model.add(Dense(256, name='L17', activation='relu'))

    model.add(Dense(2, name='L18'))
    model.compile(optimizer=Adam(lr=1e-4, beta_1=0.9, beta_2=0.999), loss='mse')

    return model

In [7]:
def get_pid_model(input_shape):
    model = Sequential()
    model.name = 'pid'

    model.add(Conv2D(32, (3, 3), input_shape=input_shape, name='L01', activation='relu'))
    model.add(Conv2D(32, (3, 3), name='L02', activation='relu'))

    model.add(AveragePooling2D(pool_size=(2, 2), name='L03'))
    model.add(Conv2D(64, (3, 3), name='L04', activation='relu'))
    model.add(Conv2D(64, (3, 3), name='L05', activation='relu'))

    model.add(AveragePooling2D(pool_size=(2, 2), name='L06'))

    model.add(Conv2D(128, (3, 3), name='L07', activation='relu'))
    model.add(Conv2D(128, (3, 3), name='L08', activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), name='L09'))

    model.add(Flatten())
    model.add(Dense(256, name='L10', activation='relu'))
    model.add(Dense(256, name='L11', activation='relu'))
    
    model.add(Dropout(0.5))
    
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

LeNet-5 as seen [here](https://www.pugetsystems.com/labs/hpc/The-Best-Way-to-Install-TensorFlow-with-GPU-Support-on-Windows-10-Without-Installing-CUDA-1187/#create-a-python-virtual-environment-for-tensorflow-using-conda).

In [8]:
def get_lenet5(input_shape):
    # building LeNet without augmentation
    model = Sequential()
    model.name = 'lenet-5'
    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape) )
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

## Importing Data

In [2]:
# getting X and y - old data
f1 = './data/sample_data/Farah_Pot_5cm_sq_15um.txt'
f2 = './data/sample_data/Grant_Pot_5cm_sq_15um.txt'
files = [f1,f2]

patch_size = 64
X, y = processing_pipeline(files, patch_size)
print(f"There are {len(y)} patches.")

There are 5193 patches.


In [3]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X,y)
# reshaping for Keras
X_train = X_train.reshape(*X_train.shape,1)
X_test = X_test.reshape(*X_test.shape,1)
# getting y arrays for Keras
n_classes = len(files)
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

## Scaling 

In [18]:
from data_processing import Scaler

In [20]:
scaler = Scaler()

In [21]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)

In [28]:
scaler = RobustScaler()

## First fit

In [94]:
# getting some parameters
input_shape = X_train.shape[1:]

In [95]:
model = get_model(input_shape)

In [96]:
# parameters for fit
batch_size = 32
epochs = 10

# fitting the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1,
          validation_data=(X_test,y_test))

Train on 3894 samples, validate on 1299 samples
Epoch 1/10
3894/3894 [==============================] - 30s 8ms/step - loss: 0.6935 - acc: 0.5128 - val_loss: 0.6935 - val_acc: 0.5089
Epoch 2/10
3894/3894 [==============================] - 28s 7ms/step - loss: 0.6906 - acc: 0.5313 - val_loss: 0.6526 - val_acc: 0.6035
Epoch 3/10
3894/3894 [==============================] - 28s 7ms/step - loss: 0.6228 - acc: 0.6328 - val_loss: 0.6045 - val_acc: 0.6713
Epoch 4/10
3894/3894 [==============================] - 28s 7ms/step - loss: 0.5791 - acc: 0.6700 - val_loss: 0.5459 - val_acc: 0.7275
Epoch 5/10
3894/3894 [==============================] - 30s 8ms/step - loss: 0.5642 - acc: 0.6898 - val_loss: 0.5323 - val_acc: 0.7306
Epoch 6/10
3894/3894 [==============================] - 30s 8ms/step - loss: 0.5503 - acc: 0.7067 - val_loss: 0.5319 - val_acc: 0.7360
Epoch 7/10
3894/3894 [==============================] - 30s 8ms/step - loss: 0.5218 - acc: 0.7337 - val_loss: 0.5125 - val_acc: 0.7460
Epoch 8

## LeNet

In [97]:
lenet = get_lenet5(input_shape)

# fitting the model
lenet.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1,
          validation_data=(X_test,y_test))

Train on 3894 samples, validate on 1299 samples
Epoch 1/10
3894/3894 [==============================] - 33s 9ms/step - loss: 0.7032 - acc: 0.6094 - val_loss: 0.6583 - val_acc: 0.6205
Epoch 2/10
3894/3894 [==============================] - 33s 8ms/step - loss: 0.5783 - acc: 0.6708 - val_loss: 0.6052 - val_acc: 0.6721
Epoch 3/10
3894/3894 [==============================] - 32s 8ms/step - loss: 0.5383 - acc: 0.7278 - val_loss: 0.6000 - val_acc: 0.6644
Epoch 4/10
3894/3894 [==============================] - 33s 8ms/step - loss: 0.4812 - acc: 0.7568 - val_loss: 0.6147 - val_acc: 0.6859
Epoch 5/10
3894/3894 [==============================] - 31s 8ms/step - loss: 0.4629 - acc: 0.7789 - val_loss: 0.6192 - val_acc: 0.6875
Epoch 6/10
3894/3894 [==============================] - 32s 8ms/step - loss: 0.3814 - acc: 0.8285 - val_loss: 0.6407 - val_acc: 0.7005
Epoch 7/10
3894/3894 [==============================] - 33s 8ms/step - loss: 0.3009 - acc: 0.8747 - val_loss: 0.7045 - val_acc: 0.7221
Epoch 8

### LeNet-5 is overfitting which we can clearly see by the acc and loss of the training data.

## Saving the model

In [20]:
# timestamp = str(int(time.time()*1000000))[0:-1]
# model.save(f'./models/{model.name}-{timestamp}.h5')

## With augmentation

In [12]:
# data augmentation
datagen = ImageDataGenerator(
    #featurewise_center = True,
    #featurewise_std_normalization=True)
    rotation_range=90,
    horizontal_flip=True,
    vertical_flip=True)
# fit it
datagen.fit(X_train)

In [ ]:
# parameters for fit
batch_size = 32
epochs = 15

# fit model with data augmentation
model.fit_generator(datagen.flow(X_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch= len(X_train)//batch_size,
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        workers=4)